In [2]:
import cv2 as cv
import tensorflow as tf
import numpy as np
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras import layers , models, optimizers
from tensorflow.keras.models import Sequential, Model
import math
from mtcnn import MTCNN

In [4]:
def boxing(img, detector):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    lis=detector.detect_faces(img)
    if len(lis)==0:
        return [0]
    i=lis[0]
    x,y,w,h=i["box"]
    gray_face = gray[y:y + h, x:x + w] 
    resized=cv.resize(gray_face, (48,48))
    image_tensor = tf.convert_to_tensor(resized, dtype=tf.float32)
    # Add dimension to match with input mode 
    image_tensor = tf.expand_dims(image_tensor, 2)
    image_tensor = tf.expand_dims(image_tensor, 0)
    image_tensor=image_tensor/255
    res=emotion_model.predict(image_tensor).argmax()
    emotion=emotion_dict[res]
    return [x,y,w,h,emotion]

In [5]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# create model structure
emotion_model = Sequential()

#emotion_model.add(data_augmentation)

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(BatchNormalization())

emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(BatchNormalization())

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(BatchNormalization())

emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(BatchNormalization())

emotion_model.add(Flatten())
emotion_model.add(Dense(256, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(BatchNormalization())

emotion_model.add(Dense(128, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(BatchNormalization())

emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# load weights into model
emotion_model.load_weights("models/emotion_model.h5")

In [6]:
detector=MTCNN()
cap = cv.VideoCapture(0)  # Use 0 for the default camera, or specify a different source if needed

while True:
    ret, frame = cap.read()  # Read a frame from the video feed
    if not ret:
        break  # If there are no more frames, exit the loop

    # Your processing code here to get the dimensions (x, y, width, height) and text
    l=boxing(frame, detector)
    if len(l)==0:
        continue
    x,y,width, height, text=l
    # Draw the rectangle on the frame
    cv.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)  # Green rectangle
    cv.putText(frame, text, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # Text above the rectangle

    # Display the frame with the overlay
    cv.imshow("Live Feed", frame)

    # Exit the loop if a key is pressed (e.g., press 'q' to quit)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv.destroyAllWindows()


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 24ms/step
